# Tester

In [1]:
import timm
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.utils.data as data
import torchvision
from tqdm import tqdm
import torch.nn.functional as F
import math

from vit_pooling import ViTPooling


gpu_ids = []
device_names = []
if torch.cuda.is_available():
    for gpu_id in range(torch.cuda.device_count()):
        gpu_ids += [gpu_id]
        device_names += [torch.cuda.get_device_name(gpu_id)]
print(gpu_ids)
print(device_names)

if len(gpu_ids) > 1:
    gpu = 'cuda:' + str(gpu_ids[1])  # GPU Number
else:
    gpu = "cuda" if torch.cuda.is_available() else "cpu"

[0, 1, 2, 3]
['NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090']


In [2]:
device = gpu
model_path = './save/ViT_timm_vit_base_patch16_224_in21k_Negative_i2012_ep8_lr0.003.pt'
TIMM_MODEL = 'vit_base_patch16_224_in21k'
BATCH_SIZE = 512
NUM_EPOCHS = 300
NUM_WORKERS = 2
LEARNING_RATE = 0.003

IMAGE_SIZE = 224
PATCH_SIZE = 16
IN_CHANNELS = 3
NUM_CLASSES = 1000
EMBED_DIM = 768
DEPTH = 12
NUM_HEADS = 12

In [3]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
# pre_train_set = torchvision.datasets.ImageFolder('./data/ImageNet-21k', transform=transform_train)
# pre_train_loader = data.DataLoader(pre_train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
train_set = torchvision.datasets.ImageFolder('../../YJ/ILSVRC2012/train', transform=transform_train)
train_loader = data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_set = torchvision.datasets.ImageFolder('../../YJ/ILSVRC2012/val', transform=transform_test)
test_loader = data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

In [4]:
class TesterTimm(object):
    def __init__(self):
        self.model = None
        self.optimizer = None
        self.scheduler = None
        self.epochs = []
        self.losses = []

    def process(self):
        self.build_model()
        self.eval_model()

    def loss_checker(self):
        self.build_model()
        print(f'Steps: {len(self.epochs)}k steps')
        [print(f'Sampling Loss: {i:.3f}') for i in self.losses]
        
    def build_model(self):
        self.model = timm.create_model(TIMM_MODEL, pretrained=True).to(device)
        checkpoint = torch.load(model_path)
        self.epochs = checkpoint['epochs']
        self.model.load_state_dict(checkpoint['model'])
        self.losses = checkpoint['losses']
        print(f'Parameter: {sum(p.numel() for p in self.model.parameters() if p.requires_grad)}')
        print(f'Classes: {self.model.num_classes}')
        print(f'Epochs: {self.epochs[-1]}')
        
    def eval_model(self):
        self.model.eval()

        correct = 0
        total = 0
        with torch.no_grad():
            for i, data in tqdm(enumerate(test_loader, 0), total=len(test_loader)):
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy {len(test_set)} test images: {100 * correct / total:.2f} %')

In [5]:
class TesterPaper(object):
    def __init__(self):
        self.model = None
        self.optimizer = None
        self.epochs = []
        self.losses = []
        self.cls_token = None

    def process(self):
        self.build_model()
        self.eval_model()
        
    def loss_checker(self):
        self.build_model()
        print(f'Steps: {len(self.epochs)}k steps')
        [print(f'Sampling Loss: {i:.3f}') for i in self.losses]

    def build_model(self):
        self.model = ViTPooling(image_size=IMAGE_SIZE,
                         patch_size=PATCH_SIZE,
                         in_channels=IN_CHANNELS,
                         num_classes=NUM_CLASSES,
                         embed_dim=EMBED_DIM,
                         depth=DEPTH,
                         num_heads=NUM_HEADS,
                         ).to(device)
        checkpoint = torch.load(model_path)
        self.epochs = checkpoint['epochs']
        self.model.load_state_dict(checkpoint['model'])
        self.losses = checkpoint['losses']
        print(f'Parameter: {sum(p.numel() for p in self.model.parameters() if p.requires_grad)}')
        print(f'Classes: {NUM_CLASSES}')
        print(f'Epochs: {self.epochs[-1]}')

    def eval_model(self):
        self.model.eval()

        correct = 0
        total = 0
        with torch.no_grad():
            for i, data in tqdm(enumerate(test_loader, 0), total=len(test_loader)):
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Accuracy {len(test_set)} test images: {100 * correct / total:.2f} %')

In [6]:
if __name__ == '__main__':
    t = TesterTimm()
    t.loss_checker()

Parameter: 102595923
Classes: 1000
Epochs: 2
Steps: 40k steps
Sampling Loss: 0.842
Sampling Loss: 0.591
Sampling Loss: 0.475
Sampling Loss: 0.835
Sampling Loss: 0.599
Sampling Loss: 0.397
Sampling Loss: 0.713
Sampling Loss: 0.374
Sampling Loss: 0.864
Sampling Loss: 0.903
Sampling Loss: 0.713
Sampling Loss: 0.761
Sampling Loss: 0.824
Sampling Loss: 0.908
Sampling Loss: 0.917
Sampling Loss: 0.398
Sampling Loss: 0.670
Sampling Loss: 0.591
Sampling Loss: 1.020
Sampling Loss: 0.796
Sampling Loss: 0.823
Sampling Loss: 0.615
Sampling Loss: 0.734
Sampling Loss: 0.691
Sampling Loss: 0.513
Sampling Loss: 0.703
Sampling Loss: 1.250
Sampling Loss: 0.759
Sampling Loss: 0.725
Sampling Loss: 0.460
Sampling Loss: 0.622
Sampling Loss: 0.637
Sampling Loss: 0.478
Sampling Loss: 0.742
Sampling Loss: 0.692
Sampling Loss: 0.625
Sampling Loss: 0.568
Sampling Loss: 0.723
Sampling Loss: 0.985
Sampling Loss: 0.934


In [7]:
if __name__ == '__main__':
    t = TesterTimm()
    t.process()

Parameter: 102595923
Classes: 1000
Epochs: 2


100%|██████████| 98/98 [07:42<00:00,  3.91s/it]

Accuracy 50000 test images: 71.24 %


In [8]:
# 프리 트레이닝
# 1스텝 = 9분
# 1에포크 = 3시간

# 파인 튜닝
# 1스텝 = 6분
# 1에포크 = 2시간

# 0.003 vs 0.0003 둘중 하나 걍 ㄱ